In [ ]:
%pip install snirf

In [ ]:
import mne
import os
from snirf import Snirf
import json
from datetime import datetime


def metadatawriter(taskdict, snirFilePath):
    with Snirf(snirFilePath, 'r') as snirf:
        file = snirf.nirs[0]
        metadata = file.metaDataTags
        # print(file.stim[0])
        print(file)
        if hasattr(metadata,'ManufacturerName'):
            SystemName = metadata.ManufacturerName
        else:
            SystemName = "NotProvided"

        if hasattr(metadata,'MeasurementDate'):
            MeasurementDate = metadata.MeasurementDate
        else:
            MeasurementDate = "NotProvided"

        if hasattr(metadata,'MeasurementTime'):
            MeasurementTime = metadata.MeasurementTime
        else:
            MeasurementTime = "NotProvided"

        if hasattr(file.stim[0],'dataLabels'):
            datalabels = file.stim[0].dataLabels
        else:
            datalabels = "NotProvided"

        if hasattr(file.stim[0],'name'):
            taskName = file.stim[0].name
        else:
            taskName = "NotProvided"

        if taskName.isnumeric():
            number = int(taskName)
            if number in taskdict:
                taskName = taskdict[number]
            else:
                print("Task association missing for ", number)
        else:
            taskName = taskName
            

        if "NotProvided" not in MeasurementDate and "NotProvided" not in MeasurementTime:
            DatetimeString = MeasurementDate + " " + MeasurementTime
            try: 
                MeasurementDateTime = datetime.strptime(DatetimeString, '%Y-%m-%d %H:%M:%S')
                matchfailed = False
            except:
                print("Date time not in format '%Y-%m-%d %H:%M:%S' trying universial time format")
                matchfailed = True

            if matchfailed:
                try:
                    MeasurementDateTime = datetime.strptime(DatetimeString, '%Y-%m-%d %H:%M:%SZ')
                except:
                    print("Date time not in format '%Y-%m-%d %H:%M:%SZ' ignoring datetime")
                    MeasurementDateTime = "NotProvided"
        else:
            MeasurementDateTime="NotProvided"

        line_items=[]
        scanjson = {
                    'SystemName': SystemName,
                    'MeasurementDate': MeasurementDate,
                    'MeasurementTime': MeasurementTime,
                    'MeasurementDateTime': MeasurementDateTime, 
                    'DataLabels': datalabels,
                    'TaskName': taskName,
                }
        line_items.append(scanjson)
        print(scanjson)
        json_object = json.dumps(scanjson, indent=4, sort_keys=True, default=str)

        with open('metadata.json', 'w') as f:
            f.write(json_object)
            print("file written")

taskMapper = {1:"fingertapping"}
filePath = '/Users/Yash/Documents/OXI_Tools/Json_writer/2022-06-21_001.snirf'

metadatawriter(taskdict=taskMapper, snirFilePath=filePath)
